In [1]:
import zipfile
import os

zip_path = "/content/Project11.zip"
extract_dir = "/content/Project11"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List the extracted files
print("📂 Extracted files:")
print(os.listdir(extract_dir))


📂 Extracted files:
['notes_on_dataset.txt', 'continuous_factory_process.csv']


In [2]:
import pandas as pd

# Load the dataset
file_path = "/content/Project11/continuous_factory_process.csv"
df = pd.read_csv(file_path)

# Show basic info and the first few rows
print("🔍 Dataset Info:")
df.info()

print("\n📈 First 5 Rows:")
df.head()


🔍 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14088 entries, 0 to 14087
Columns: 116 entries, time_stamp to Stage2.Output.Measurement14.U.Setpoint
dtypes: float64(108), int64(7), object(1)
memory usage: 12.5+ MB

📈 First 5 Rows:


,time_stamp,AmbientConditions.AmbientHumidity.U.Actual,AmbientConditions.AmbientTemperature.U.Actual,Machine1.RawMaterial.Property1,Machine1.RawMaterial.Property2,Machine1.RawMaterial.Property3,Machine1.RawMaterial.Property4,Machine1.RawMaterialFeederParameter.U.Actual,Machine1.Zone1Temperature.C.Actual,Machine1.Zone2Temperature.C.Actual,...,Stage2.Output.Measurement10.U.Actual,Stage2.Output.Measurement10.U.Setpoint,Stage2.Output.Measurement11.U.Actual,Stage2.Output.Measurement11.U.Setpoint,Stage2.Output.Measurement12.U.Actual,Stage2.Output.Measurement12.U.Setpoint,Stage2.Output.Measurement13.U.Actual,Stage2.Output.Measurement13.U.Setpoint,Stage2.Output.Measurement14.U.Actual,Stage2.Output.Measurement14.U.Setpoint
0,2019-03-06 10:52:33,17.24,23.53,11.54,200,963.0,247,1241.26,72.0,72.3,...,0.0,7.93,0.0,5.65,0.0,1.85,0.0,2.89,0.0,11.71
1,2019-03-06 10:52:34,17.24,23.53,11.54,200,963.0,247,1246.09,72.0,72.3,...,0.0,7.93,0.0,5.65,0.0,1.85,0.0,2.89,0.0,11.71
2,2019-03-06 10:52:35,17.24,23.53,11.54,200,963.0,247,1246.29,72.0,72.3,...,0.0,7.93,0.0,5.65,0.0,1.85,0.0,2.89,0.0,11.71
3,2019-03-06 10:52:36,17.24,23.53,11.54,200,963.0,247,1247.59,72.0,72.3,...,0.0,7.93,0.0,5.65,0.0,1.85,0.0,2.89,0.0,11.71
4,2019-03-06 10:52:37,17.24,23.53,11.54,200,963.0,247,1252.83,72.1,72.4,...,0.0,7.93,0.0,5.65,0.0,1.85,0.0,2.89,0.0,11.71


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# Drop timestamp
df_cleaned = df.drop(columns=["time_stamp"])

# Identify target columns
stage1_targets = [col for col in df_cleaned.columns if "Stage1.Output.Measurement" in col and ".U.Actual" in col]
stage2_targets = [col for col in df_cleaned.columns if "Stage2.Output.Measurement" in col and ".U.Actual" in col]
all_targets = stage1_targets + stage2_targets

# Features
X = df_cleaned.drop(columns=all_targets)

# Target sets
y_stage1 = df_cleaned[stage1_targets]
y_stage2 = df_cleaned[stage2_targets]

# Train-test split
X_train, X_test, y1_train, y1_test = train_test_split(X, y_stage1, test_size=0.2, random_state=42)
_, _, y2_train, y2_test = train_test_split(X, y_stage2, test_size=0.2, random_state=42)


In [8]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
import time

# Define Models
model_stage1 = MultiOutputRegressor(XGBRegressor(n_estimators=150, learning_rate=0.1, max_depth=6, random_state=42))
model_stage2 = MultiOutputRegressor(XGBRegressor(n_estimators=150, learning_rate=0.1, max_depth=6, random_state=42))

# Train Stage 1
start = time.time()
model_stage1.fit(X_train, y1_train)
print("✅ Stage 1 trained in", round(time.time() - start, 2), "seconds")

# Train Stage 2
start = time.time()
model_stage2.fit(X_train, y2_train)
print("✅ Stage 2 trained in", round(time.time() - start, 2), "seconds")


✅ Stage 1 trained in 19.63 seconds
✅ Stage 2 trained in 16.75 seconds


In [9]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Predict
y1_pred = model_stage1.predict(X_test)
y2_pred = model_stage2.predict(X_test)

# Evaluate
r2_stage1 = r2_score(y1_test, y1_pred)
rmse_stage1 = np.sqrt(mean_squared_error(y1_test, y1_pred))

r2_stage2 = r2_score(y2_test, y2_pred)
rmse_stage2 = np.sqrt(mean_squared_error(y2_test, y2_pred))

# Display
print("📌 Stage 1 - R²:", round(r2_stage1, 4), "| RMSE:", round(rmse_stage1, 4))
print("📌 Stage 2 - R²:", round(r2_stage2, 4), "| RMSE:", round(rmse_stage2, 4))


📌 Stage 1 - R²: 0.704 | RMSE: 1.754
📌 Stage 2 - R²: 0.7326 | RMSE: 1.8303
